<a href="https://colab.research.google.com/github/Dr-Mohamed-Elkholy/Machine-Learning-Projects/blob/main/Deep_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Classification with Deep Neural Networks
You should build an end-to-end machine learning pipeline using a deep learning model. In particular, you should do the following:
- Load the `fashion mnist` dataset from [TensorFlow](https://www.tensorflow.org/tutorials/keras/classification). The loaded dataset is already split into training and test sets.
- Build an end-to-end machine learning pipeline, including a [deep learning](https://www.tensorflow.org/tutorials/keras/classification) model.
- Optimize your pipeline by validating your design decisions.
- Test the best pipeline on the test set and report various [evaluation metrics](https://scikit-learn.org/0.15/modules/model_evaluation.html).  
- Check the documentation to identify the most important hyperparameters, attributes, and methods of the model. Use them in practice.

## Loading the data

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

In [ ]:


# Loading the Fashion MNIST dataset
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()


class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# # Normalizing the images to the range of [0., 1.]
train_images = train_images / 255.0
test_images = test_images / 255.0

# Checking the shape of the datasets
print(train_images.shape, train_labels.shape, test_images.shape, test_labels.shape)


(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


#Train the model

In [ ]:
# Step 2: Reshape the data to add a channel dimension
train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

# Step 3: Build the deep learning model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)), # to identify pattern like edges and textures
    tf.keras.layers.MaxPooling2D((2, 2)), # condense or shrink images to 2 * 2
    tf.keras.layers.Flatten(), # flatten to line of numberss
    tf.keras.layers.Dense(128, activation='relu'), # interprete the pattern
    tf.keras.layers.Dense(10, activation='softmax') # like rethink or reclassify
])

# Step 4: Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Model summary to check the structure of the neural network
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 13, 13, 32)        0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 5408)              0         
                                                                 
 dense_2 (Dense)             (None, 128)               692352    
                                                                 
 dense_3 (Dense)             (None, 10)                1290      
                                                                 
Total params: 693962 (2.65 MB)
Trainable params: 693962 (2.65 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

# Fitting

In [ ]:
# Step 5: Train the model
from tensorflow.keras.callbacks import EarlyStopping

# Create early stopping (stop training after the validation loss has not improved for 5 epochs)
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Include early stopping in the model's fit method
history = model.fit(train_images, train_labels, epochs=10, validation_split=0.2, callbacks=[early_stopping])


Epoch 1/10
1500/1500 [==============================] - 39s 26ms/step - loss: 0.4098 - accuracy: 0.8545 - val_loss: 0.3116 - val_accuracy: 0.8915
Epoch 2/10
1500/1500 [==============================] - 36s 24ms/step - loss: 0.2795 - accuracy: 0.9001 - val_loss: 0.2758 - val_accuracy: 0.9005
Epoch 3/10
1500/1500 [==============================] - 37s 24ms/step - loss: 0.2326 - accuracy: 0.9149 - val_loss: 0.2707 - val_accuracy: 0.9005
Epoch 4/10
1500/1500 [==============================] - 35s 23ms/step - loss: 0.1989 - accuracy: 0.9267 - val_loss: 0.2521 - val_accuracy: 0.9103
Epoch 5/10
1500/1500 [==============================] - 37s 25ms/step - loss: 0.1723 - accuracy: 0.9361 - val_loss: 0.2631 - val_accuracy: 0.9063
Epoch 6/10
1500/1500 [==============================] - 38s 26ms/step - loss: 0.1461 - accuracy: 0.9463 - val_loss: 0.2494 - val_accuracy: 0.9144
Epoch 7/10
1500/1500 [==============================] - 39s 26ms/step - loss: 0.1274 - accuracy: 0.9530 - val_loss: 0.2750 -

# Evaulation

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_images, test_labels)

# Display the test set accuracy
print("Accuracy on test set:", test_accuracy)


313/313 [==============================] - 2s 6ms/step - loss: 0.5920 - accuracy: 0.9045
Accuracy on test set: 0.9045000076293945


# Metrics

In [ ]:


# Predict the values from the test dataset
test_predictions = model.predict(test_images)
# Convert predictions classes to one hot vectors
predicted_classes = np.argmax(test_predictions, axis=1)

# compute the confusion matrix
confusion_mtx = confusion_matrix(test_labels, predicted_classes)



# compute classification report
classification_report = classification_report(test_labels, predicted_classes, target_names=class_names)

print("Confusion Matrix:\n", confusion_mtx)
print("\nClassification Report:\n", classification_report)


313/313 [==============================] - 4s 13ms/step
Confusion Matrix:
 [[898   1  21  10   4   1  62   1   2   0]
 [  2 978   1   8   9   0   1   0   1   0]
 [ 22   1 882   5  49   0  41   0   0   0]
 [ 37   4   8 880  37   0  31   0   3   0]
 [  2   1 101  14 830   1  51   0   0   0]
 [  2   0   0   0   0 976   0  14   0   8]
 [143   1  59  19  69   0 701   1   7   0]
 [  0   0   0   0   0   5   0 966   0  29]
 [  6   1   5   4   3   2   2   3 973   1]
 [  2   0   0   0   2   8   1  26   0 961]]

Classification Report:
               precision    recall  f1-score   support

 T-shirt/top       0.81      0.90      0.85      1000
     Trouser       0.99      0.98      0.98      1000
    Pullover       0.82      0.88      0.85      1000
       Dress       0.94      0.88      0.91      1000
        Coat       0.83      0.83      0.83      1000
      Sandal       0.98      0.98      0.98      1000
       Shirt       0.79      0.70      0.74      1000
     Sneaker       0.96      0.97   